In [71]:
#Impory the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
import pickle

In [72]:
#load data
originalDataset = pd.read_csv('news_articles.csv')
textData = originalDataset.iloc[:,10]
fakeOrReal = originalDataset.iloc[:,8]
fakeOrReal = fakeOrReal.replace(['Real'],1)
fakeOrReal = fakeOrReal.replace(['Fake'],0)
dataset = pd.concat([textData, fakeOrReal], axis=1, sort=False)
dataset.drop(dataset.tail(50).index,inplace=True)

In [73]:
dataset.head(3)

,text_without_stopwords,label
0,print pay back money plus interest entire fami...,1.0
1,attorney general loretta lynch plead fifth bar...,1.0
2,red state fox news sunday reported morning ant...,1.0


In [74]:
#creating bag of words
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 150000)
X = cv.fit_transform(dataset.iloc[:,0]).toarray()
y = dataset.iloc[:,-1]

In [75]:
# Splitting the dataset into the Training set and Test set
# separar aleatoriamente o dataset em base de treino e teste
# separar 70% dos registros para treino e 30% para testes 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

In [76]:
from sklearn.ensemble import ExtraTreesClassifier
# criação do modelo Árvode de decisão
modelo = ExtraTreesClassifier( criterion='gini')
# executar modelo ou treinar o modelo de ML ou ainda aprender as relações dos dados
modelo.fit(X_train,y_train)

ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=100,
                     n_jobs=None, oob_score=False, random_state=None, verbose=0,
                     warm_start=False)

In [77]:
# Acurácia do modelo
# y_teste estão as classificações corretas 
# score = compara os dados de testes com os dados de treino (modelo rodou para os dados de treino) 
resultado = modelo.score(X_test, y_test)
print('Acurácia =', resultado)

Acurácia = 0.8268292682926829


In [78]:
y_pred = modelo.predict(X_test)

In [79]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[260,  13],
       [ 58,  79]], dtype=int64)

In [80]:
# Making the Detail Confusion Matrix
print(pd.crosstab(y_test, y_pred, rownames=['Reais'],colnames=['Preditos'], margins=True))

Preditos  0.0  1.0  All
Reais                  
0.0       260   13  273
1.0        58   79  137
All       318   92  410


In [81]:
filename = 'model18_extrd1.sav'
pickle.dump(modelo, open(filename, 'wb'))

In [82]:
#move threshold
threshold = 0.425

predicted_proba = modelo.predict_proba(X_test)
predicted = (predicted_proba [:,1] >= threshold).astype('int')

cm = confusion_matrix(y_test, predicted)
cm

array([[230,  43],
       [ 35, 102]], dtype=int64)